In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import os
import matplotlib
import matplotlib.pyplot as plt
import librosa
import soundfile
import keras
import librosa.display
import IPython.display as ipd

In [ ]:
df = pd.read_csv('/home/birdsong/kaggle-data/train_tp.csv')
df = df.sample(frac=1).reset_index(drop=True) #permutes the rows of the dataframe


In [ ]:
df1 = df[0:int(len(df)*0.8)] #0.8 vs 0.2 train-test split
df1_test = df[int(len(df)*0.8)::]

## Augment train data by including multiple random 2-second clips containing bird song (8 times the train data)

In [ ]:
#load train_tp.csv into a pandas df
#loop through each row of the df (supposedly inefficient, but seems necessary here)
spectrograms = []
probabilities = []
labels = []
labels_fp_are_tp = []
frog_or_bird = []
for i in range(8):
    for index, row in df1.iterrows():
        infile =  '/home/birdsong/kaggle-data/train/%s.flac' %row['recording_id']
        outfile = '/home/birdsong/processed_data/augmented_clips_tp/%d.flac' %index 
        sig, sr = librosa.load(infile)
        tmin = row['t_min']*sr 
        tmax = row['t_max']*sr
    #os.system('clear')
    #print(index)
        clip_length = 2 #No. of seconds
        l = len(sig)
        if tmax - tmin < clip_length*sr:
            if  tmax - clip_length*sr > 0:
                upper = np.random.randint(tmax, min(tmin + clip_length*sr,l-1)) 
                lower = upper - clip_length*sr
            else:
                lower = np.random.randint(0, tmin)
                upper = lower + clip_length*sr
        else:
        #pick first clip_length seconds
        #lower = 0 
        #upper = clip_length*sr
        #or pick middle segment
            a = np.random.uniform(low = 0.1, high = 0.9)
            lower = round(tmin + (tmax - tmin - clip_length*sr)*a)
            upper = round(tmax - (tmax - tmin - clip_length*sr)*(1-a))
        sig = sig[lower:upper]
        #create mel spectrograms and labels
        mel_spectrogram = librosa.feature.melspectrogram(y = sig, sr=sr, n_fft=2048, hop_length=1024, n_mels=64,  fmin= 64, fmax = 16384)
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
        #print(log_mel_spectrogram.shape)
        spectrograms.append(log_mel_spectrogram.reshape(64*44))
        prob = np.zeros(24, dtype = np.float16)
        prob[row["species_id"]] = 1
        labels.append(str(row["species_id"])+str(row["songtype_id"]))
        labels_fp_are_tp.append(str(row["species_id"])+str(row["songtype_id"]))
        if row["species_id"] in [0, 1, 2, 3, 4, 8, 12, 14, 16, 18, 21]:
            frog_or_bird.append('frog')
        else:
            frog_or_bird.append('bird')
        
        probabilities.append(prob)
        print(i,index)
    #Uncomment to use moving average filter
    #window = 70
    #post_sig = np.asarray([(sum(sig[y:y+window])/window) for y in range(len(sig)-window)])
    
    # uncomment to actually write files
    #soundfile.write(outfile, sig, sr)


In [ ]:
X_train = np.array(spectrograms)
p_train = np.array(probabilities)
y_train = np.array(labels)

In [ ]:
pd.DataFrame(X_train).to_csv('/home/birdsong/processed_data/augmented_clips_tp/spectrograms_hidef.csv')
pd.DataFrame(y_train).to_csv('/home/birdsong/processed_data/augmented_clips_tp/labels.csv')
pd.DataFrame(p_train).to_csv('/home/birdsong/processed_data/augmented_clips_tp/probabilities.csv')
pd.DataFrame(np.array(frog_or_bird)).to_csv('/home/birdsong/processed_data/augmented_clips_tp/frog_or_bird.csv')


In [ ]:
X_train.shape

In [ ]:
y_train.shape

## Create spectrograms for test data

In [ ]:
test_spectrograms = []
test_probabilities = []
test_labels = []
test_labels_fp_are_tp = []
test_frog_or_bird = []

for index, row in df2.iterrows():
    infile =  '/home/birdsong/kaggle-data/train/%s.flac' %row['recording_id']
    outfile = '/home/birdsong/processed_data/augmented_clips_tp/%d.flac' %index 
    sig, sr = librosa.load(infile)
    tmin = row['t_min']*sr 
    tmax = row['t_max']*sr
    #os.system('clear')
    #print(index)
    clip_length = 2 #No. of seconds
    l = len(sig)
    if tmax - tmin < clip_length*sr:
        if  tmax - clip_length*sr > 0:
            upper = np.random.randint(tmax, min(tmin + clip_length*sr,l-1)) 
            lower = upper - clip_length*sr
        else:
            lower = np.random.randint(0, tmin)
            upper = lower + clip_length*sr
    else:
        #pick first clip_length seconds
        #lower = 0 
        #upper = clip_length*sr
        #or pick middle segment
        a = np.random.uniform(low = 0.1, high = 0.9)
        lower = round(tmin + (tmax - tmin - clip_length*sr)*a)
        upper = round(tmax - (tmax - tmin - clip_length*sr)*(1-a))
    sig = sig[lower:upper]
    
    mel_spectrogram = librosa.feature.melspectrogram(y = sig, sr=sr, n_fft=2048, hop_length=1024, n_mels=64,  fmin= 64, fmax = 16384)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    #print(log_mel_spectrogram.shape)
    test_spectrograms.append(log_mel_spectrogram.reshape(64*44))
    prob = np.zeros(24, dtype = np.float16)
    prob[row["species_id"]] = 1
    test_labels.append(str(row["species_id"])+str(row["songtype_id"]))
    test_labels_fp_are_tp.append(str(row["species_id"])+str(row["songtype_id"]))
    if row["species_id"] in [0, 1, 2, 3, 4, 8, 12, 14, 16, 18, 21]:
        test_frog_or_bird.append('frog')
    else:
        test_frog_or_bird.append('bird')
        
    test_probabilities.append(prob)
    #print(i,index)

## Run Classifier

In [ ]:
y_new = np.array(frog_or_bird)

In [ ]:
X_test = np.array(test_spectrograms)
p_test = np.array(test_probabilities)
y_test = np.array(test_labels)
y_new_test = np.array(test_frog_or_bird)

In [ ]:
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
#X_train, X_test, y_train, y_test = train_test_split(X, y_frog_or_bird, test_size = 0.25, random_state = 792)

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
#reg = LogisticRegression(max_iter=1000).fit(X_train_scaled, y_train)


reg = LogisticRegression(
    penalty="l1", solver = "saga", random_state = 25
).fit(X_train_scaled, y_train)

In [ ]:
X_test_scaled = scaler.transform(X_test)
y_pred = reg.predict(X_test_scaled)

In [ ]:
correct = np.count_nonzero(y_test == y_pred)
all = len(y_test)
correct, all, correct/all

In [ ]:
y_train_pred = reg.predict(X_train_scaled)

In [ ]:
correct = np.count_nonzero(y_train == y_train_pred)
all = len(y_train)
correct, all, correct/all